In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

### 1. 데이터 로드 및 전처리

In [2]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:03<00:00, 49273764.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### 2. Alexnet 모델 정의 (PyTorch 사용)

In [3]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features  = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d((6, 6))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

### 3. 모델 학습 설정

In [4]:
# 3. 모델 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AlexNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 4. 모델 학습

In [5]:
num_epochs = 20
patience = 2  # Early stopping patience
best_loss = float('inf')
stopping_step = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        # 기존의 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 값 출력
        running_loss += loss.item()
        if i % 100 == 99:    # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

        # Early stopping 조건 확인
    if running_loss < best_loss:
        best_loss = running_loss
        stopping_step = 0
    else:
        stopping_step += 1
        if stopping_step >= patience:
            print('Early stopping triggered')
            break

print('학습 완료')

[Epoch 1, Batch 100] loss: 2.011
[Epoch 1, Batch 200] loss: 1.641
[Epoch 1, Batch 300] loss: 1.488
[Epoch 2, Batch 100] loss: 1.252
[Epoch 2, Batch 200] loss: 1.206
[Epoch 2, Batch 300] loss: 1.133
[Epoch 3, Batch 100] loss: 0.999
[Epoch 3, Batch 200] loss: 0.962
[Epoch 3, Batch 300] loss: 0.928
[Epoch 4, Batch 100] loss: 0.808
[Epoch 4, Batch 200] loss: 0.824
[Epoch 4, Batch 300] loss: 0.803
[Epoch 5, Batch 100] loss: 0.690
[Epoch 5, Batch 200] loss: 0.690
[Epoch 5, Batch 300] loss: 0.695
[Epoch 6, Batch 100] loss: 0.592
[Epoch 6, Batch 200] loss: 0.623
[Epoch 6, Batch 300] loss: 0.620
[Epoch 7, Batch 100] loss: 0.510
[Epoch 7, Batch 200] loss: 0.550
[Epoch 7, Batch 300] loss: 0.558
[Epoch 8, Batch 100] loss: 0.453
[Epoch 8, Batch 200] loss: 0.484
[Epoch 8, Batch 300] loss: 0.506
[Epoch 9, Batch 100] loss: 0.407
[Epoch 9, Batch 200] loss: 0.428
[Epoch 9, Batch 300] loss: 0.441
[Epoch 10, Batch 100] loss: 0.346
[Epoch 10, Batch 200] loss: 0.375
[Epoch 10, Batch 300] loss: 0.406
[Epoch 

### 5. 모델 평가

In [6]:
# 5. 테스트 정확도 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'최종 테스트 정확도: {100 * correct / total:.4f}%')

최종 테스트 정확도: 76.5400%
